In [1]:
import z3
import pandas as pd
import pprint
import timeit
import numpy as np

#----------------------------------------------------------------
#   Functions
#----------------------------------------------------------------
def a2nVars(appList, nodeList):
    '''Builds an m x n pandas datafram with apps and rows(index) and nodes as columns'''
    a2n = [ [ z3.Real("a%s%s" % (i+1, j+1)) 
         for j in range(len(nodeList)) ]
       for i in range(len(appList))]
    
    a2n_df = pd.DataFrame(a2n, index=appList, columns=nodeList)
    
    return a2n_df

def DependsOn(a2n_df, app, deps):
    #res = [z3.Implies(a2n_df.loc[app].sum()>0, a2n_df.loc[dep].sum()>0) for dep in deps]
    #print("res %s" %res)
    return(z3.And([z3.Implies(a2n_df.loc[app].sum()>0, a2n_df.loc[dep].sum()>0) for dep in deps]))

def setA2NDomain(a2n_df):
    a2n_domain = [z3.Or(a2n_df.loc[app, node]==0, a2n_df.loc[app, node]==1) \
                  for app in a2n_df.index \
                  for node in a2n_df.columns]
    return a2n_domain

def setRsrcConstraint(a2n_df, rpa_df, rpn_df):
    app_Load = rpa_df.dot(a2n_df)
    #display(app_Load)
    node_Surplus = rpn_df - app_Load
    
    rsrc_constraint = [app_Load.loc[rsrc,node]>=0 \
                       for rsrc in rpa_df.index \
                       for node in a2n_df.columns]
    return rsrc_constraint

def setMDDConstraint(a2a_df, a2n_df):
    mdd = list()
    for app in a2n_df.index:
        if a2a_df.loc[app].sum() > 0: #only put in the constraint if the app has dependencies
            deps = [dep for dep in a2a_df.loc[app].index if a2a_df.loc[app,dep]==1]
            mdd.append(DependsOn(a2n_df, app, deps))
    return mdd
    

def setConstraints(a2a_df, a2n_df, rpa_df, rpn_df):
    '''adds constraints to the values in an matrix'''
    a2n_domain = setA2NDomain(a2n_df)       
    rsrc_constraint = setRsrcConstraint(a2n_df, rpa_df, rpn_df)    
    mdd = setMDDConstraint(a2a_df, a2n_df)    
    return (a2n_domain + rsrc_constraint + mdd)

def getNetworkUtil(a2n_df, rpa_df, rsrc):
    '''Returns network utilization. Can potentially be used for balancing later. All pieces are available'''
    # sum 1 to m a_ij*c
    node_load = rpa_df.dot(a2n_df) #pandas dataFrame dot operator
    
    #R(a)
    def getNetworkLoad(node_load, rsrc):
        '''Returns a z3.ArithRef of a sum of all apps on all nodes'''
        network_load = (node_load.loc[rsrc].values).sum()
        return network_load
    
    network_load = getNetworkLoad(node_load, rsrc)    
    network_supply = rpn_df.loc[rsrc].sum()
    node_util = node_load.divide(rpn_df)
    network_util = network_load/(network_supply)
    return network_util

def latencyCost(a2a_df, a2n_df):
    numNodes = len(a2n_df.columns) # number of FSSNs
    markings = list() # The equations for each possible placement of the dependencies
    #list of all apps that have dependencies
    hasDeps = [node for node in a2n_df.index if a2a_df.loc[node].sum()>0]

    #n nodes for primary app
    #2^(n)-1 placements for a particular dependency
    #should be 2^(n)-1 x n x sum of all dependencies
    #eg. n=5 app1 has 2 deps app2 has 1 dep
    # count = 5 x 31 x 3 = 465
    for srcNode in a2n_df.columns:
        #print("new sources")
        for pr_app in hasDeps:
            if a2n_df.loc[pr_app, srcNode] ==1: #app that has deps is on srcNode
                ## ANOTHER FOR LOOP IF THERE ARE MULTIPLE DEPENDENCIES
                primaryDeps = [dep for dep in a2n_df.index if a2a_df.loc[pr_app, dep]==1]
                for dep in primaryDeps:
                    #print("new dep")
                    # iterate through the 2^numNodes potential dependency placements
                    # and save them as binary representation to look up when running the optimizer
                    for i in range(2**numNodes):
                        mark = format(i, 'b').zfill(numNodes) # express placment in binary    
                        #print(mark)
                        # The dependency must be place somewhere. So this skips the case where i=0
                        # This may not be necessary thanks to the constraint. 
                        if mark == format(0,'b').zfill(numNodes):
                            continue
                        marking = list() # list for placment equations e.g (1 - a11)*(1 - a12)*a13
                        latency = [] # latencies between node hosting primary app and all nodes hosting the dependency for a particular placement. 
                        #print ("-----NEW MARKING-----")
                        # iterate through nodes using the binary placement to generate marking equation
                        for i, node in enumerate(a2n_df.columns):
                            #print ("mark %s" %mark)
                            #print ("i %s " %i)
                            #print ("mark[i] %s" %type(int(mark[i]))) 
                            # add marking to list and if binary has a 1, add latency to list of possible latencies
                            if int(mark[i]) == 1:
                                marking.append(a2n_df.loc[dep, node]) #a_ij
                                latency.append(n2n_df.loc[srcNode, node])
                                #latency.append(random.randint(0,10))
                            if int(mark[i]) == 0:
                                marking.append(1 - a2n_df.loc[dep,node]) # 1- a_ij
                        #print("latency %s" %latency)                    
                        #print("min latency %s" %min(latency).item())
                        #print("min latency %s" %min(latency))
                        #print("type(min(latency)) = %s" %type(min(latency)))
                        #print("marking: %s" %marking)
                        #print("marking * latency: %s" %(z3.Product(marking) * min(latency).item()))
                        #print("marking * latency: %s" %(z3.Product(marking) * min(latency)))
                        #markings.append(z3.Product(marking) * min(latency).item())
                        markings.append(z3.Product(marking) * min(latency))
    return markings

def scaleLatency(a2a_df, numNodes, max_edge_latency):
        min_latency = 0 #app is deployed everywhere
        #max_latency = max([sum(self.n2n_df.loc[node, :]) for node in self.nodeList]) #app is deployed on node with highest total latency
        max_deps = max([(a2a_df.loc[app]).sum() for app in a2a_df.columns])
        max_latency = max_edge_latency * (numNodes-1) * max_deps
        scale_latency = 1/(max_latency - min_latency)
        return scale_latency

def a2nEval(a2n_df, model):
    a2n_df_r = a2n_df.copy()
    #deps = [dep for pr in a2n_df.index for dep in a2n_df.index if a2a_df.loc[pr, dep]==1]
    for app in a2n_df_r.index:
        for node in a2n_df.columns:
            if(type (a2n_df_r.loc[app,node]) == z3.ArithRef):
                a2n_df_r.loc[app, node] = model.eval(a2n_df_r.loc[app, node])
    return a2n_df_r
    

def getF(a2a_df, a2n_df, rpa_df, p, q, sym):
    #display(a2n_df_r)
    network_util_r = z3.simplify(getNetworkUtil(a2n_df, rpa_df, 'ram'))
    latency_r = z3.Sum(latencyCost(a2a_df, a2n_df))
    numNodes = len(a2n_df.columns)
    scale_latency = scaleLatency(a2a_df, numNodes, 10) #numpy.float64
    f = p*network_util_r + (1-p)*latency_r*scale_latency
    
    if sym == False:
        f_r = float(z3.simplify(f).as_decimal(512))
        print("f = p*network_util_r + (1-p)*latency_r*scale_latency")
        print( "f = " + str(p) + " * " + network_util_r.as_decimal(10) + " + " + 
                        str("%.2f" %(1.0-p))+ " * " + z3.simplify(latency_r * scale_latency).as_decimal(5))
        print("f = " +  z3.simplify(p*network_util_r).as_decimal(5) + "  +  " + 
                        z3.simplify((1-p)*latency_r*scale_latency).as_decimal(10))
    else:
        f_r = z3.simplify(f)
    return f_r

In [2]:
def iterSat(a2a_df, a2n_df, rpa_df, p, q, iters):
    sat_loops = 0
    max_loops = iters
    s = z3.Solver()   
    constraints = setConstraints(a2a_df, a2n_df, rpa_df, rpn_df)
    s.add(constraints)
    objF = z3.Real("objF")
    f = getF(a2a_df, a2n_df, rpa_df, p, q, sym=True)
    s.add(objF == f)
    
    start = timeit.default_timer()
    
    #f_old = -1
    while (s.check() == z3.sat and sat_loops<max_loops):
        m = s.model()
        #display(m)    
        if(sat_loops>0):
            s.pop()
        s.push()
        s.add(objF<m[objF])
        sat_loops = sat_loops+1       
    stop = timeit.default_timer()
    print ("sat_loops = %s" %sat_loops)
    print ("run time: %s" %(stop - start))
    f_r = m[objF].as_decimal(5)
    a2n_df_r = a2nEval(a2n_df, m)
    return(a2n_df_r, f_r)

In [3]:
def optimize(a2a_df, a2n_df, rpa_df, rpn_df, p, q):
    opt = z3.Optimize()       
    constraints = setConstraints(a2a_df, a2n_df, rpa_df, rpn_df)
    opt.add(constraints)      
    objF = z3.Real("objF")
    f = getF(a2a_df, a2n_df, rpa_df, p, q, sym=True)  
    opt.add(objF == f)    
    
    start = timeit.default_timer()
    opt.minimize(objF)
    
    sat = opt.check()
    opt_model = opt.model()
    #print("optimization model \n %s \n" %opt_model)
    stop = timeit.default_timer()
    print ("run time: %s" %(stop - start))
    
    f_r = opt_model[objF].as_decimal(5) 
    a2n_df_r = a2nEval(a2n_df, opt_model)        
    return a2n_df_r, f_r, 
    

In [4]:
import operator
def bruteForce(placeable, a2a_df, a2n_df, rpa_df, rpn_df, p , q):    
    numVars = len(placeable.values.flatten())#a_ijs in numpy array from pandas dataframe excludes initial placement
    my_list = list()
    solutions = list()
    
    s = z3.Solver()       
    constraints = setConstraints(a2a_df, a2n_df, rpa_df, rpn_df)
    s.add(constraints)
    objF = z3.Real("objF")
    f = getF(a2a_df, a2n_df, rpa_df, p, q, sym=True)
    s.add(objF == f)    
    
    start = timeit.default_timer()
    
    for i in range(2**numVars):
        marking = format(i, 'b').zfill(numVars) 
        m = 0
        s.push()
        for app in placeable.index:
            for node in placeable.columns:
                s.add(a2n_df.loc[app,node] == z3.RealSort().cast(marking[m]))
                m = m + 1  
        #check placement for feasibility
        issat = s.check()
        #print(issat)
        if (issat == z3.sat):
            last_model = s.model()
            f_r = last_model[objF].as_decimal(5)
            #print(f_r)
            my_list.append(f_r)
            solutions.append(last_model)
            #if len(solutions)>10:
            #    break                
        s.pop()

    stop = timeit.default_timer()    
    print ("run time: %s" %(stop - start))
    
    max_index, max_value = max(enumerate(my_list), key=operator.itemgetter(1))
    min_index, min_value = min(enumerate(my_list), key=operator.itemgetter(1))

    min_solution = solutions[min_index]
    
    f_r = min_solution[objF].as_decimal(5)
    a2n_df_r = a2nEval(a2n_df, min_solution)
    return(a2n_df_r, f_r)    


In [5]:
#----------------------------------------------------------------
#          GIVENS
#----------------------------------------------------------------
appList = sorted(['APP1', 'APP2', 'APP3'])
nodeList = sorted(['FSSN3', 'FSSN2', 'FSSN1', 'FSSN4', 'FSSN5'])
resourceTypes = ('storage', 'ram')#('cpu', 'ram', 'storage')
# ----- APPS x Resources -------
rpa = [[2048, 1024, 1024],
       [1024, 512, 512]]
rpa_df = pd.DataFrame(rpa, index=resourceTypes, columns=appList)

rpn = [[ 8192,  8192, 16384,  8192,  8192],
       [ 4096,  4096,  4096,  4096,  4096]]
rpn_df = pd.DataFrame(rpn, index=resourceTypes, columns=nodeList)

# Rows depend on columns
a2a = [[0, 1, 1],
       [0, 0, 0],
       [0, 0, 0]]
a2a_df = pd.DataFrame(a2a, index=appList, columns=appList) 

n2n = [[0, 3, 4, 5, 6],
       [3, 0, 5, 6, 7],
       [4, 5, 0, 7, 8],
       [5, 6, 7, 0, 9],
       [6, 7, 8, 9, 0]]
n2n_df = pd.DataFrame(n2n, index=nodeList, columns=nodeList) 

#Using a function for now, but that won't be the case in the end. 
def initial_deployment(a2n_df):
        a2n_df.loc['APP1'] = 1
        placeable = a2n_df.drop(['APP1'])
        return a2n_df, placeable
    
#----------------------------------------------------------------
#         Derived
#----------------------------------------------------------------
# make list of dependencies
a2n_df_raw = a2nVars(appList, nodeList)
#display(a2n_df)
a2n_df, placeable = initial_deployment(a2n_df_raw.copy())

p = 0.7
q = 0

In [6]:
#rpa_df.loc['ram', "APP2"] = 512
#rpa_df.loc['ram', "APP3"] = 512
#rpn_df.loc['ram', 'FSSN2'] = 14096
#----------------------------------------------------------------
#        Use
#----------------------------------------------------------------
a2n_df_r, f_r = bruteForce(placeable, a2a_df, a2n_df, rpa_df, rpn_df, p , q)
print("brute min cost found: %s" %f_r)
display(a2n_df_r)

a2n_df_r, f_r = iterSat(a2a_df, a2n_df, rpa_df, p, q, iters=100)   
print("itersat found: %s " %f_r)
display(a2n_df_r)


a2n_df_r, f_r = optimize(a2a_df, a2n_df, rpa_df, rpn_df, p, q)
print("Optimize found: %s " %f_r)
display(a2n_df_r)

run time: 3.2003325650002807
brute min cost found: 0.33250?


,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,0,1,1
APP3,1,0,0,1,1


sat_loops = 8
run time: 1.051304345999597
itersat found: 0.33250? 


,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,0,1,1
APP3,1,0,0,1,1


run time: 7.512598756999068
Optimize found: 0.33250? 


,FSSN1,FSSN2,FSSN3,FSSN4,FSSN5
APP1,1,1,1,1,1
APP2,1,0,0,1,1
APP3,1,0,0,1,1
